In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import pytorch_lightning as pl
import torch

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics
from mlp_model import MoleculeDataset, HiModel

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/drd2/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/drd2/test_1.csv', index_col=0)

train

,smiles,value
383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...
5444,O=C1c2ccccc2C(=O)N1CCCCN1CCCN(C(c2ccccc2)c2ccc...,True
4391,O=C(CCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
4397,O=C(CCCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
5999,OC12C3C4CC5C6C4C1C6C(C53)N2CC1CCCCC1,False


In [4]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]
train_fps = np.array(train_fps, dtype=np.float32)

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]
test_fps = np.array(test_fps, dtype=np.float32)


In [5]:
train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

In [6]:
train_dataset[0][0].shape

(167,)

In [7]:
param_dict = {
    'layers': [
        [167, 32, 32],
        [167, 16, 16],
        [167, 32],
        [167, 8, 4],
        [167, 4]
    ],
    'use_dropout': [True],
    'dropout': [0.0, 0.0, 0.2, 0.4, 0.6],
    'lr': [0.01],
    'l2': [0.0, 0.0, 0.001, 0.005, 0.01],
    'batch_size': [32]
}

In [8]:
from sklearn.model_selection import ParameterSampler
from pytorch_lightning.callbacks import EarlyStopping

hyperparam_list = list(ParameterSampler(param_dict, n_iter=30))

best_score = 0.0
best_hyperparam = None

for i, hyperparam in enumerate(hyperparam_list):
    early_stop_callback = EarlyStopping(
        monitor='test_prc_auc',  # the metric to monitor
        min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
        patience=20,  # number of epochs with no improvement after which training will be stopped
        verbose=False,  # verbosity mode
        mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
    )

    print('HYPERPARAMETER', i)
    print(hyperparam)

    train_dataloader = DataLoader(train_dataset, batch_size=hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=hyperparam['batch_size'], shuffle=False)

    model = HiModel(hyperparam)
    wandb.init(project='drd2_hi_maccs')
    wandb_logger = WandbLogger(project='drd2_hi_maccs')
    trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback])
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)
    score = early_stop_callback.best_score.detach().cpu().numpy()
    if score > best_score:
        best_score = score
        best_hyperparam = hyperparam

HYPERPARAMETER 0
{'use_dropout': True, 'lr': 0.01, 'layers': [167, 16, 16], 'l2': 0.001, 'dropout': 0.2, 'batch_size': 32}
Sequential(
  (0): Linear(in_features=167, out_features=16, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: steshinss. Use `wandb login --relogin` to force relogin


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 3.0 K 
-------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
best_score

array(0.7236294, dtype=float32)

In [11]:
best_hyperparam

{'use_dropout': True,
 'lr': 0.01,
 'layers': [167, 16, 16],
 'l2': 0.01,
 'dropout': 0.4,
 'batch_size': 32}

In [6]:
best_hyperparam = {
    'use_dropout': True,
    'lr': 0.01,
    'layers': [167, 16, 16],
    'l2': 0.01,
    'dropout': 0.4,
    'batch_size': 32
    }

In [9]:
from pytorch_lightning.callbacks import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor='test_prc_auc',  # the metric to monitor
    min_delta=0.00,  # minimum change in the monitored quantity to qualify as an improvement
    patience=20,  # number of epochs with no improvement after which training will be stopped
    verbose=False,  # verbosity mode
    mode='max'  # 'min' for minimizing the metric, 'max' for maximizing
)
train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

model = HiModel(best_hyperparam)
wandb.init(project='drd2_hi_maccs')
wandb_logger = WandbLogger(project='drd2_hi_maccs')
trainer = pl.Trainer(max_epochs=150, logger=wandb_logger, accelerator="gpu", devices=[1], default_root_dir=None, callbacks=[early_stop_callback])
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

Sequential(
  (0): Linear(in_features=167, out_features=16, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: steshinss. Use `wandb login --relogin` to force relogin


/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 3.0 K 
-------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [10]:
all_preds = []
all_true = []
with torch.no_grad():
    for _, batch in enumerate(test_dataloader):
        X, y = batch
        y_pred = model(X)
        all_preds.append(y_pred)
        all_true.append(y)

In [11]:
all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
all_true = np.squeeze(torch.cat(all_true).cpu().numpy())

In [12]:
get_hi_metrics(pd.DataFrame({'value': all_true}), all_preds)

{'roc_auc': 0.4955610376018539,
 'bedroc': 0.47442827418749317,
 'prc_auc': 0.6089207475452082}

# Final Evaluation

In [13]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]
    train_fps = np.array(train_fps, dtype=np.float32)

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]
    test_fps = np.array(test_fps, dtype=np.float32)

    train_dataset = MoleculeDataset(train_fps, np.array(train['value'], dtype=np.float32))
    test_dataset = MoleculeDataset(test_fps, np.array(test['value'], dtype=np.float32))

    train_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)

    model = HiModel(best_hyperparam)
    trainer = pl.Trainer(max_epochs=31, accelerator="gpu", devices=1, default_root_dir=None)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

    train_result = train.copy()
    train_predict_dataloader = DataLoader(train_dataset, batch_size=best_hyperparam['batch_size'], shuffle=False)
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(train_predict_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    train_result['preds'] = all_preds

    test_result = test.copy()
    all_preds = []
    with torch.no_grad():
        for _, batch in enumerate(test_dataloader):
            X, _ = batch
            y_pred = model(X)
            all_preds.append(y_pred)
    all_preds = np.squeeze(torch.cat(all_preds).cpu().numpy())
    test_result['preds'] = all_preds

    return train_result, test_result

In [14]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/drd2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/drd2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/drd2/mlp_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/drd2/mlp_maccs/test_{i}.csv')


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /data/steshin/gero_benchmark/notebooks/models/hi/drd2/lightning_logs


Sequential(
  (0): Linear(in_features=167, out_features=16, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.4, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 3.0 K 
-------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=31` reached.
